# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [13]:
# Intput File (CSV)
# input_data_file = "output_data/cities.csv"
my_input_data_file = "../WeatherPy/output_data/my_cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
my_csvDF = pd.read_csv(my_input_data_file)
my_csvDF.head(10)

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID
0,Chui,-33.6971,-53.4616,18.71,76,98,3.29,UY,1615001307,0
1,Coos Bay,43.3665,-124.2179,6.43,81,90,4.63,US,1615001233,1
2,Wenling,28.3669,121.3600,12.78,91,100,5.36,CN,1615001124,2
3,Cidreira,-30.1811,-50.2056,21.58,87,7,4.20,BR,1615001372,3
4,Trat,12.5000,102.5000,30.28,74,20,1.03,TH,1615001372,4
5,Port Elizabeth,-33.9180,25.5701,20.00,88,0,4.63,ZA,1615001372,5
6,Marsabit,2.3284,37.9899,18.83,58,98,5.77,KE,1615001372,6
7,Ushuaia,-54.8000,-68.3000,11.00,71,90,5.66,AR,1615001372,7
8,Ribeira Grande,38.5167,-28.7000,15.33,79,62,5.50,PT,1615001372,8
9,Port Alfred,-33.5906,26.8910,16.67,92,0,1.34,ZA,1615001372,9


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
